In [10]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [11]:
%reload_ext autoreload
%autoreload 2

In [12]:
from vaegan3 import VAE
from vaegan3 import NetD
from vaegan3 import Aux
from vaegan3 import loss_function

In [13]:
bsz = 100
criterion = nn.BCELoss()


In [14]:
train_loader = torch.utils.data.DataLoader(
    datasets.Omniglot('../data', download=True,
                   transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.Omniglot('../data', background=True, transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)


Files already downloaded and verified


In [15]:
netG = VAE()
netD = NetD()
aux = Aux()

In [16]:
optimizerD = optim.Adam(netD.parameters(), lr=1e-4)
optimizerG = optim.Adam(netG.parameters(), lr=1e-4)
optimizer_aux = optim.Adam(aux.parameters(), lr=1e-4)

In [17]:
input = torch.FloatTensor(bsz,105,105)
label = torch.FloatTensor(bsz)
real_label=1
fake_label=0
USE_CUDA=1

if(USE_CUDA):
    netG=netG.cuda()
    netD=netD.cuda()
    aux = aux.cuda()
    criterion=criterion.cuda()
    input,label=input.cuda(), label.cuda()

In [18]:
%reload_ext autoreload
%autoreload 2
for epoch in range(100):
    for i, (data,y) in enumerate(train_loader):
        gamma = 1.0
        real_cpu = data;

        real_cpu = real_cpu.cuda()
        y=y.cuda()
        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(bsz).fill_(real_label)

        inputv = Variable(input)
        labelv = Variable(label)
        y = Variable(y)

        #need variables for dis
        #x_l, x_l_tilde
        
        
        #do discriminator calculations
        netD.zero_grad()
        #fc3_weight,fc4_weight = aux.return_weights()
        mu,logvar = netG(inputv, y)
        std = logvar.mul(0.5).exp_()
        eps = Variable(std.data.new(std.size()).normal_())
        z=eps.mul(std).add_(mu)
        fake = aux(z, y)
        
        x_l_tilde, output_fake = netD(fake)
        x_l, output_real = netD(inputv)
        #x_l_aux, output_fake_aux = netD(fake_aux)
        L_GAN_real = criterion(output_real, labelv)
        L_GAN_real.backward(retain_graph=True)
        
        labelv = Variable(label.fill_(fake_label))
        L_GAN_fake = criterion(output_fake, labelv)
        L_GAN_fake.backward(retain_graph=True)
        
        z_p = Variable(std.data.new(std.size()).normal_())
        fake_aux = aux(z_p, y)
        x_l_aux, output_aux = netD(fake_aux)
        L_GAN_aux = criterion(output_aux,labelv)
        L_GAN_aux.backward(retain_graph=True)
        optimizerD.step()
           
        
        #get weights of netG and use in aux
        aux.zero_grad()
        labelv=Variable(label.fill_(real_label))
        
        L_dec_vae = gamma*loss_function(x_l_tilde,x_l,mu,logvar)
        L_dec_fake = criterion(output_fake,labelv)
        L_dec_aux  = criterion(output_aux,labelv)
        L_dec_vae.backward(retain_graph=True)
        L_dec_fake.backward(retain_graph=True)
        L_dec_aux.backward(retain_graph=True)
        optimizer_aux.step()
        
        

        #encoder loss 
        netG.zero_grad()
        L_enc = loss_function(x_l_tilde, x_l,mu,logvar)
        L_enc.backward()
        optimizerG.step()

    
        if i % 100 == 0:
            print('real_cpu.size()', real_cpu.size(), "iteration: ", i)
            vutils.save_image(real_cpu,
                            './results/real_samples2.png',
                                normalize=True)
            vutils.save_image(fake.data.view(-1,1,105,105),
                                './results/fake_samples2.png',
                                normalize=True)
    if epoch % 10 == 0:
        print('epoch: ', epoch)
        vutils.save_image(fake.data.view(-1,1,105,105),
                                './results/fake_samples2_{0}.png'.format(epoch+100),
                                normalize=True)

AssertionError: 

In [ ]:
torch.save(netG, 'netG.pth')
torch.save(netD, 'netD.pth')
torch.save(aux, 'aux.pth')

## Load Pretrained Model

In [133]:
netG = torch.load('netG.pth')
netD = torch.load('netD.pth')
aux = torch.load('aux.pth')

### Apply an alpha to change the label of a sample batch

In [157]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)

In [219]:
data, y = iter(test_loader).next()
vutils.save_image(data.view(-1,1,28,28),
                                './fake.png',
                                normalize=True)

In [220]:
%reload_ext autoreload
%autoreload 2

# change the label to whatever number you want to turn the images to (in this case it is 8) and the alpha (.5)
mu,logvar = netG(Variable(data).cuda(), Variable(y).cuda(), Variable(torch.tensor([8])).cuda(), .5)
std = logvar.mul(0.5).exp_()
eps = Variable(std.data.new(std.size()).normal_())
z=eps.mul(std).add_(mu)
fake = aux(z, y, Variable(torch.tensor([8])).cuda(), .5)
vutils.save_image(fake.data.view(-1,1,28,28),
                                './generated2.png',
                                normalize=True)

In [221]:
mu,logvar = netG(Variable(fake), Variable(y).cuda())
std = logvar.mul(0.5).exp_()
eps = Variable(std.data.new(std.size()).normal_())
z=eps.mul(std).add_(mu)
fake2 = aux(z, y)
vutils.save_image(fake2.data.view(-1,1,28,28),
                                './generated3.png',
                                normalize=True)

In [222]:
mu,logvar = netG(Variable(data).cuda(), Variable(y).cuda(), Variable(torch.tensor([8])).cuda(), 1)
std = logvar.mul(0.5).exp_()
eps = Variable(std.data.new(std.size()).normal_())
z=eps.mul(std).add_(mu)
fake = aux(z, y, Variable(torch.tensor([8])).cuda(), 1)
vutils.save_image(fake.data.view(-1,1,28,28),
                                './generated.png',
                                normalize=True)